In [15]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from keras.utils import to_categorical
# Tu código inicial para la preparación de datos y la unión de DataFrames
import pandas as pd
from sklearn.metrics import r2_score, mean_absolute_error

archivos = ['cluster_0.csv', 'cluster_1.csv', 'cluster_2.csv', 'cluster_3.csv']

dataframes = []

for i, archivo in enumerate(archivos):
    df = pd.read_csv(archivo)
    df['Cluster'] = i
    
    # Procesamiento adicional para crear la columna 'Conteo Dias' basada en la fecha
    df['FECHA'] = pd.to_datetime(df['FECHA'])
    start_date = df['FECHA'].min()
    df['DiasDesdeInicio'] = (df['FECHA'] - start_date).dt.days
    df = df.drop(columns=['FECHA'])
    dias_desde_inicio = df['DiasDesdeInicio']
    df = df.drop(columns=['DiasDesdeInicio'])
    df.insert(0, 'Conteo Dias', dias_desde_inicio)
    
    dataframes.append(df)

df_unido = pd.concat(dataframes)
df_unido.reset_index(drop=True, inplace=True)


fallas = {
    8:  "OUT0210-Válvula Entrada Producto Cerrada por Falta Consentimiento de Centralita C.I.P.",
    11: "Error Acoplam. Enjuagadora",
    12: "Falta Comunicación PLC Depósito",
    21: "IN0054-Señal Ftc Presencia Botella Caída",
    22: "INA0228- Avería Sonda Presión Sondra",
    30: "INA0227-Vacío Bajo Demasiado Alto",
    31: "IN0031-Grasa Bloqueada" ,
    32: "INA0200-Nivel Máximo Depósito",
    77: "Puertas Desbloq.",
    86:  "IN0086-Control Sobretemperatura Armario",
    112: "INA0227-Vacío Bajo Demasiado Bajo",
    124: "IN0577-Control Botella Colgada",
    129: "Error Acoplamiento Tapadora 2",
    145: "IN0064-Señal Mesa Acum. Bot. Llena",
    237: "Botella Rota TV",
    285: "OUT0207-Presión Cilindros Demasiado Baja",
    286:  "Máquina Desfasada",
    293: "INA0000-Falta Ref. Velocidad de Inversor Motor Principal"
}

df_unido['EsFalla'] = False

for codigo, descripcion in fallas.items():
    if descripcion in df_unido.columns:
        df_unido['EsFalla'] = df_unido['EsFalla'] | (df_unido[descripcion] > 0)

In [ ]:
def procesar_y_graficar(df, cluster_id):
    # Preprocesamiento
    df['FECHA'] = pd.to_datetime(df['FECHA'])
    start_date = df['FECHA'].min()
    df['DiasDesdeInicio'] = (df['FECHA'] - start_date).dt.days
    df = df.drop(columns=['FECHA'])
    dias_desde_inicio = df['DiasDesdeInicio']
    df = df.drop(columns=['DiasDesdeInicio'])
    df.insert(0, 'Conteo Dias', dias_desde_inicio)

    # Identificar columnas de fallas
    columnas_fallas = [fallas[codigo] for codigo in fallas if fallas[codigo] in df.columns]
    eventos_no_fallas = [col for col in df.columns if col not in columnas_fallas]
    resultados_prueba_totales = []
    for evento in eventos_no_fallas:
        print(f"Procesando evento: {evento} en el cluster {cluster_id}")

        # Preparar datos para esta evento
        X = df.drop(columns=[evento]).values
        y = df[evento].values
        dates = df['Conteo Dias'].values

        # División de datos
        sequence_length = 10
        X_train, X_test, y_train, y_test, dates_train, dates_test = train_test_split(X, y, dates, test_size=0.2, random_state=42)

        # Preparar datos para RNN
        sequence_length = 10  # Longitud de la secuencia

          # Seleccionar las columnas para el entrenamiento (excluyendo la columna objetivo)
        X = df.drop(columns=[evento]).values

          # La columna objetivo es la que queremos predecir
        y = df[evento].values

          # Obtener las fechas correspondientes a los datos
        dates = df['Conteo Dias'].values

          # Dividir los datos en conjuntos de entrenamiento y prueba
        X_train, X_test, y_train, y_test, dates_train, dates_test = train_test_split(X, y, dates, test_size=0.2, random_state=42)


        # Construcción y entrenamiento del modelo RNN
        X_train_rnn = []
        y_train_rnn = []
        dates_train_rnn = []

        for i in range(len(X_train) - sequence_length):
            X_train_rnn.append(X_train[i:i + sequence_length])
            y_train_rnn.append(y_train[i + sequence_length])
            dates_train_rnn.append(dates_train[i + sequence_length])

        X_train_rnn = np.array(X_train_rnn)
        y_train_rnn = np.array(y_train_rnn)

        X_test_rnn = []
        y_test_rnn = []
        dates_test_rnn = []

        for i in range(len(X_test) - sequence_length):
            X_test_rnn.append(X_test[i:i + sequence_length])
            y_test_rnn.append(y_test[i + sequence_length])
            dates_test_rnn.append(dates_test[i + sequence_length])

        X_test_rnn = np.array(X_test_rnn)
        y_test_rnn = np.array(y_test_rnn)
        X_train_rnn = X_train_rnn.astype('float32')
        X_test_rnn = X_test_rnn.astype('float32')

        model = tf.keras.Sequential([
          tf.keras.layers.SimpleRNN(64, activation='relu', input_shape=(sequence_length, X_train_rnn.shape[2])),
          tf.keras.layers.Dense(64, activation='relu'),
          tf.keras.layers.Dense(1)  # No se usa la función de activación para problemas de regresión
        ])

# 4. Compilar el modelo
        model.compile(optimizer='adam', loss='mean_squared_error')

        #early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# 5. Entrenar el modelo y guardar historial
        history = model.fit(X_train_rnn, y_train_rnn, epochs=160, batch_size=120, validation_data=(X_test_rnn, y_test_rnn))

# Evaluar el modelo en el conjunto de prueba
        loss = model.evaluate(X_test_rnn, y_test_rnn)
        print(f'Pérdida en el conjunto de prueba: {loss}')

# Graficar pérdida y la validación
        plt.figure(figsize=(12, 4))
        plt.subplot(1, 2, 1)
        plt.plot(history.history['loss'], label='Pérdida de entrenamiento')
        plt.plot(history.history['val_loss'], label='Pérdida de validación')
        plt.xlabel('Epochs')
        plt.ylabel('Pérdida')
        plt.title('Pérdida de entrenamiento y validación')
        plt.legend()

        plt.tight_layout()
        plt.show()

        # Realizar predicciones en el conjunto de entrenamiento
        y_train_pred = model.predict(X_train_rnn)
        resultados_prueba_totales.append(y_train_pred)

# Crear un array de índices de 'DiasDesdeInicio' para el eje x
        indices = np.arange(len(dates_train_rnn))

# Graficar las predicciones y la ocurrencia real en función de 'DiasDesdeInicio'
        plt.figure(figsize=(12, 6))
        plt.plot(indices, y_train_pred, label='Predicción', color='r', linestyle='--')
        plt.plot(indices, y_train_rnn, label='Ocurrencia Real', color='b', linestyle='-')
        plt.xlabel('DiasDesdeInicio')
        plt.ylabel('Valor Predicho')
        plt.title(f'Predicción vs. Ocurrencia Real en Entrenamiento para {evento}')
        plt.legend()
        plt.grid(True)

# Puedes personalizar más el gráfico, como ajustar etiquetas de ejes, títulos, colores, etc.
        plt.show()

        # Evaluación del modelo y gráficos
        r2 = r2_score(y_train_rnn, y_train_pred)
        print(f'Coeficiente de Determinación (R²): {r2}')
        mae = mean_absolute_error(y_train_rnn, y_train_pred)
        print(f'Error Absoluto Medio (MAE): {mae}')


    return resultados_prueba_totales


# Lista de archivos de cluster
archivos_clusters = ['cluster_0.csv', 'cluster_1.csv', 'cluster_2.csv', 'cluster_3.csv']

resultados_clusters = []

for archivo in archivos_clusters:
    df_cluster = pd.read_csv(archivo)
    resultados_prueba_cluster = procesar_y_graficar(df_cluster, archivo.split('.')[0])  # Obtenemos solo resultados de prueba
    
    # Convertir las listas de resultados de prueba en un DataFrame
    df_resultados_prueba_cluster = pd.DataFrame(np.hstack(resultados_prueba_cluster))

    # Obtener el ID del cluster del nombre del archivo
    cluster_id = archivo.split('_')[1].split('.')[0]  # Obtener el ID del cluster del nombre del archivo

    # Obtener las descripciones o nombres de los eventos
    columnas_fallas = [fallas[codigo] for codigo in fallas if fallas[codigo] in df_cluster.columns]
    eventos_no_fallas = [col for col in df_cluster.columns if col not in columnas_fallas]

    # Agregar los nombres de los eventos como la primera fila del DataFrame


    df_resultados_prueba_cluster.columns = eventos_no_fallas[1:]

    # Guardar los resultados en un archivo CSV para el cluster actual
    df_resultados_prueba_cluster.to_csv(f'resultados_prueba_cluster_{cluster_id}.csv', index=False)

In [11]:
import pandas as pd
df1 = pd.read_csv('cluster_0.csv')
df2 = pd.read_csv('resultados_prueba_cluster_0.csv')

df2.shape

df2['FECHA'] = df1['FECHA'].head(163)
df2 = df2.drop('DiasDesdeInicio', axis=1)
df2.head(3)

ultima_columna = df2.columns[-1]

# Crear una lista con el nuevo orden de las columnas
nuevo_orden_columnas = [ultima_columna] + [col for col in df2.columns if col != ultima_columna]

# Reorganizar las columnas según el nuevo orden
df2 = df2[nuevo_orden_columnas]

df2.to_csv(f'resultados_cluster_{0}.csv', index=False)

In [12]:
import pandas as pd
df1 = pd.read_csv('cluster_1.csv')
df2 = pd.read_csv('resultados_prueba_cluster_1.csv')

df2.shape
df2['FECHA'] = df1['FECHA'].head(163)
df2 = df2.drop('DiasDesdeInicio', axis=1)
df2.head(3)
ultima_columna = df2.columns[-1]

# Crear una lista con el nuevo orden de las columnas
nuevo_orden_columnas = [ultima_columna] + [col for col in df2.columns if col != ultima_columna]

# Reorganizar las columnas según el nuevo orden
df2 = df2[nuevo_orden_columnas]
df2.to_csv(f'resultados_cluster_{1}.csv', index=False)
df2.head(3)

,FECHA,IN0470-Control Estrella Entrada,IN0517-Control Caja Compresión Abierta,IN0070-Señal Bloqueo Inversor Motor Principal,Estrellas Enjuagadora,Faltan Corchos en Botellas.x,IN0250-Control Presostato Atascamiento Filtro 1 Campana,IN0294-Control Auto. Bomba Vacío Alto,IN0574-Control Protecciones Delanteras Abiertas,IN0575-Control Protecciones Traseras Abiertas,Mando Manual Habilitado,IN0291-Control Auto. Motor Ajuste Depósito,CIP-Señal de CIP de Alarma,IN0476-Control Protecciones Laterales Abiertas
0,2022-09-06,75.473045,2.056876,0.639284,-0.017290,0.328427,0.203364,0.172391,-0.084234,19.926374,0.400925,0.157283,-0.193803,0.034101
1,2022-09-07,89.668785,2.833339,0.072972,-0.238601,0.289631,0.065853,-0.061351,-0.356879,27.980007,-0.225899,0.159842,0.065415,-0.075193
2,2022-09-08,61.767895,-0.020598,-0.001077,5.265711,2.659300,-0.118472,0.093786,0.256764,2.912547,0.119572,8.486596,-0.069412,0.016575


In [13]:
import pandas as pd
df1 = pd.read_csv('cluster_2.csv')
df2 = pd.read_csv('resultados_prueba_cluster_2.csv')

df2.shape
df2['FECHA'] = df1['FECHA'].head(163)
df2 = df2.drop('DiasDesdeInicio', axis=1)
df2.head(3)
ultima_columna = df2.columns[-1]

# Crear una lista con el nuevo orden de las columnas
nuevo_orden_columnas = [ultima_columna] + [col for col in df2.columns if col != ultima_columna]

# Reorganizar las columnas según el nuevo orden
df2 = df2[nuevo_orden_columnas]
df2.to_csv(f'resultados_cluster_{2}.csv', index=False)

In [14]:
import pandas as pd
df1 = pd.read_csv('cluster_3.csv')
df2 = pd.read_csv('resultados_prueba_cluster_3.csv')

df2.shape
df2['FECHA'] = df1['FECHA'].head(163)
df2 = df2.drop('DiasDesdeInicio', axis=1)
df2.head(3)
ultima_columna = df2.columns[-1]

# Crear una lista con el nuevo orden de las columnas
nuevo_orden_columnas = [ultima_columna] + [col for col in df2.columns if col != ultima_columna]

# Reorganizar las columnas según el nuevo orden
df2 = df2[nuevo_orden_columnas]
df2.to_csv(f'resultados_cluster_{3}.csv', index=False)

In [27]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from keras.utils import to_categorical
# Tu código inicial para la preparación de datos y la unión de DataFrames
import pandas as pd
from sklearn.metrics import r2_score, mean_absolute_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

import pandas as pd
df = pd.read_csv('resultados_cluster_2.csv')
df2 = pd.read_csv("cluster_2.csv")
seq_length = 212

# Specify the columns you want to plot
columns_no_falla = [
    'Señal Falta Solicitud lista de CIP',
    'Mantenimiento Programado',
    'Conteo Botellas Alcanzado',
    'IN0173-Control Rosca Salida'
]

nuevo_df = df[['Señal Falta Solicitud lista de CIP', 'Mantenimiento Programado','Conteo Botellas Alcanzado','IN0173-Control Rosca Salida']]

falla = df2[['OUT0210-Válvula Entrada Producto Cerrada por Falta Consentimiento de Centralita C.I.P.']]
falla1 = falla.iloc[:163]

# Crear el modelo de la red neuronal
modelo = Sequential()
modelo.add(LSTM(64, input_shape=(4, 1)))
modelo.add(Dense(1, activation='sigmoid'))  # Capa de salida con activación sigmoid para predecir la probabilidad

# Compilar el modelo
modelo.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Entrenar el modelo
modelo.fit(nuevo_df, falla1, epochs=10, batch_size=32)

# Realizar predicciones
predicciones = modelo.predict(nuevo_df)


print(predicciones[-5:])

Epoch 1/10
6/6 [==============================] - 4s 8ms/step - loss: 0.7327 - accuracy: 0.1718
Epoch 2/10
6/6 [==============================] - 0s 7ms/step - loss: -0.5730 - accuracy: 0.0859
Epoch 3/10
6/6 [==============================] - 0s 7ms/step - loss: -2.0054 - accuracy: 0.0859
Epoch 4/10
6/6 [==============================] - 0s 7ms/step - loss: -3.6042 - accuracy: 0.0859
Epoch 5/10
6/6 [==============================] - 0s 7ms/step - loss: -5.4335 - accuracy: 0.0859
Epoch 6/10
6/6 [==============================] - 0s 8ms/step - loss: -8.0858 - accuracy: 0.0859
Epoch 7/10
6/6 [==============================] - 0s 8ms/step - loss: -11.4961 - accuracy: 0.0859
Epoch 8/10
6/6 [==============================] - 0s 8ms/step - loss: -15.0148 - accuracy: 0.0859
Epoch 9/10
6/6 [==============================] - 0s 8ms/step - loss: -19.2467 - accuracy: 0.0859
Epoch 10/10
6/6 [==============================] - 1s 3ms/step
[[0.9770038 ]
 [0.99998856]
 [0.9999889 ]
 [0.9999897 ]
 [0.99

In [29]:
import pandas as pd
df2 = pd.read_csv('cluster_1.csv')
df = pd.read_csv('resultados_cluster_1.csv')
seq_length = 212

# Specify the columns you want to plot

nuevo_df = df[['Mando Manual Habilitado', 'IN0291-Control Auto. Motor Ajuste Depósito', 'IN0476-Control Protecciones Laterales Abiertas', 'IN0574-Control Protecciones Delanteras Abiertas']]

falla = df2[['Máquina Desfasada']]
falla1 = falla.iloc[:163]

# Crear el modelo de la red neuronal
modelo = Sequential()
modelo.add(LSTM(64, input_shape=(4, 1)))
modelo.add(Dense(1, activation='sigmoid'))  # Capa de salida con activación sigmoid para predecir la probabilidad

# Compilar el modelo
modelo.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Entrenar el modelo
modelo.fit(nuevo_df, falla1, epochs= 200, batch_size= 256)

# Realizar predicciones
predicciones = modelo.predict(nuevo_df)


print(predicciones[-5:])

Epoch 1/200
1/1 [==============================] - 3s 3s/step - loss: 0.6936 - accuracy: 0.2025
Epoch 2/200
1/1 [==============================] - 0s 12ms/step - loss: 0.6916 - accuracy: 0.4969
Epoch 3/200
1/1 [==============================] - 0s 12ms/step - loss: 0.6896 - accuracy: 0.6626
Epoch 4/200
1/1 [==============================] - 0s 17ms/step - loss: 0.6876 - accuracy: 0.7362
Epoch 5/200
1/1 [==============================] - 0s 13ms/step - loss: 0.6856 - accuracy: 0.7546
Epoch 6/200
1/1 [==============================] - 0s 17ms/step - loss: 0.6836 - accuracy: 0.8037
Epoch 7/200
1/1 [==============================] - 0s 17ms/step - loss: 0.6815 - accuracy: 0.8344
Epoch 8/200
1/1 [==============================] - 0s 14ms/step - loss: 0.6793 - accuracy: 0.8405
Epoch 9/200
1/1 [==============================] - 0s 17ms/step - loss: 0.6770 - accuracy: 0.8466
Epoch 10/200
1/1 [==============================] - 0s 15ms/step - loss: 0.6747 - accuracy: 0.8466
Epoch 11/200
1/1 [===